In [2]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, roc_auc_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.initializers import GlorotUniform

In [3]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

labels = pd.read_csv('labels.csv')

In [4]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/Line/AI&Robotics_2019-01-15 00:00:00_14.png',
    'images/Line/AI&Robotics_2019-01-24 00:00:00_30.png',
    'images/Line/AI&Robotics_2019-03-28 00:00:00_90.png',
    'images/Line/AI&Robotics_2019-06-20 00:00:00_180.png', 
    'images/Line/AI&Robotics_2019-12-26 00:00:00_365.png'   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

80 100 85 120
80 100 132 120
80 100 226 120
80 100 414 120
80 100 602 120


In [5]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [6]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_7673/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [7]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [8]:
# Divide data based on timeframes
labels_dict = {
    "14" : labels[labels['Image'].str.contains('_14_')],
    "30" : labels[labels['Image'].str.contains('_30_')],
    "90" : labels[labels['Image'].str.contains('_90_')],
    "180" : labels[labels['Image'].str.contains('_180_')],
    "365" : labels[labels['Image'].str.contains('_365_')]
    }

In [9]:
for key in labels_dict:
    # Convert the Image_Array column to a numpy array
    labels_dict[key]['Image_Array'] = np.array(labels_dict[key]['Image_Array']) / 255.0
    # Shuffle the DataFrame
    #labels_dict[key] = shuffle(labels_dict[key], random_state=42).reset_index(drop=True)


/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_7673/2776856819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_dict[key]['Image_Array'] = np.array(labels_dict[key]['Image_Array']) / 255.0


In [10]:
def create_cnn_model(num_blocks, input_shape=()):
    model = Sequential()
    
    current_shape = input_shape
    for i in range(num_blocks):
        # Add Convolutional layer
        filters = 64 * (2 ** i)
        model.add(Conv2D(filters, kernel_size=(5, 3), dilation_rate=(1, 1), padding='same',
                         input_shape=current_shape))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.01))
        model.add(MaxPooling2D(pool_size=(2, 1)))

        # Update current_shape for the next layer
        current_shape = (current_shape[0] // 2, current_shape[1] // 3, filters)

    # Calculate the number of neurons for the fully connected layer
    flatten_size = current_shape[0] * current_shape[1] * current_shape[2]
    
    # Flatten the output of the last CNN block
    model.add(Flatten())

    # Fully connected layer with Dropout
    model.add(Dense(flatten_size, activation='relu', kernel_initializer=GlorotUniform()))
    model.add(Dropout(0.5))

    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [11]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Set dilation rate and strides
        strides = (1, 3)

        # Add Convolutional layer
        model.add(Conv2D(filters, kernel_size=(5, 3), strides=strides, padding='same',
                         input_shape=input_shape if i == 0 else model.output_shape[1:]))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer only if dimensions allow
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))  # Example Dense layer, adjust as needed
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Adjust the final layer as per your classification/regression needs

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Train and evaluate the model for each img_type, timeframe, and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(4, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(4, input_shape=(120, 414, 1))
                elif timeframe == 365:
                    model = create_cnn_model(4, input_shape=(120, 602, 1))
                
                X = np.array(data['Image_Array'].tolist())
                lastPrice = np.array(data['LastPrice'].tolist())
                futurePrice = np.array(data['FuturePrice'].tolist())
                y = data['Label']
                # Split the data into training and validation sets in temporal order
                split_index = int(len(X) * 0.7)
                X_train, X_test = X[:split_index], X[split_index:]
                y_train, y_test = y[:split_index], y[split_index:]
                lastPrice_test = lastPrice[split_index:]
                futurePrice_test = futurePrice[split_index:]
                
                # Train the model on 10 windows of data
                window_size = len(X_train)/10
                num_windows = 10

                for epoch in range(10):  # 10 epochs
                    start = 0
                    end = int(window_size * (epoch + 1))
                    X_temp = X_train[start:end]
                    y_temp = y_train[start:end]
                    split_index = int(len(X) * 0.9)
                    X_train_temp, X_val_temp = X_temp[:split_index], X_temp[split_index:]
                    y_train_temp, y_val_temp = y_temp[:split_index], y_temp[split_index:]
                    model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp))            
                
                # Evaluate the model on test data
                y_pred = model.predict(X_test)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)
                
                accuracy = accuracy_score(y_test, y_pred_binary)
                precision = precision_score(y_test, y_pred_binary)
                recall = recall_score(y_test, y_pred_binary)
                f1_score = fbeta_score(y_test, y_pred_binary, beta=1)

                y_test_array = y_test.values.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                # Calculate Rate of Return RoR
                ror = np.where(y_pred_binary == 1, ((futurePrice_test - lastPrice_test)/lastPrice_test) * 100, 0)
                # Drop all 0s from the array
                ror = ror[ror != 0]
                # Calculate average RoR
                avg_ror = np.mean(ror)
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average RoR: {avg_ror}")
                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Average_RoR': avg_ror
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation_scores.csv'.")

In [16]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(3, input_shape=(120, 132, 1))
            elif timeframe == 90:
                model = create_cnn_model(4, input_shape=(120, 226, 1))
            elif timeframe == 180:
                model = create_cnn_model(4, input_shape=(120, 414, 1))
            elif timeframe == 365:
                model = create_cnn_model(4, input_shape=(120, 602, 1))
            
            X = np.array(data['Image_Array'].tolist())
            lastPrice = np.array(data['LastPrice'].tolist())
            futurePrice = np.array(data['FuturePrice'].tolist())
            y = data['Label']
            # Split the data into training and validation sets in temporal order
            split_index = int(len(X) * 0.7)
            X_train, X_test = X[:split_index], X[split_index:]
            y_train, y_test = y[:split_index], y[split_index:]
            lastPrice_test = lastPrice[split_index:]
            futurePrice_test = futurePrice[split_index:]
            
            window_size = len(X_train)/10
            num_windows = 10

            for epoch in range(10):  # 10 epochs
                start = 0
                end = int(window_size * (epoch + 1))
                X_temp = X_train[start:end]
                y_temp = y_train[start:end]
                split_index = int(len(X) * 0.9)
                X_train_temp, X_val_temp = X_temp[:split_index], X_temp[split_index:]
                y_train_temp, y_val_temp = y_temp[:split_index], y_temp[split_index:]
                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp))            
            
            # Evaluate the model
            y_pred = model.predict(X_test)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)
            
            accuracy = accuracy_score(y_test, y_pred_binary)
            precision = precision_score(y_test, y_pred_binary)
            recall = recall_score(y_test, y_pred_binary)
            f1_score = fbeta_score(y_test, y_pred_binary, beta=1)

            y_test_array = y_test.values.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            # Calculate Rate of Return RoR
            ror = np.where(y_pred_binary == 1, ((futurePrice_test - lastPrice_test)/lastPrice_test) * 100, 0)
            # Drop all 0s from the array
            ror = ror[ror != 0]
            # Calculate average RoR
            avg_ror = np.mean(ror)
            
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            print(f"Hit Rate: {hit_rate}")
            print(f"Average RoR: {avg_ror}")
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Image_Type': img_type,
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Average_RoR': avg_ror
            }, ignore_index=True)
            
            # Save the model
            model.save(f"models/combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_model_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_model_evaluation_scores.csv'.")

Predicting 5 days ahead with 14 days timeframe.
17/17 [==============================] - 0s 22ms/step
Evaluation Metrics:
Accuracy: 0.5968992248062015
Precision: 0.7142857142857143
Recall: 0.569620253164557
F1 Score: 0.6338028169014085
Hit Rate: 0.5968992248062015
Average RoR: 0.3098982981250379
Model saved as combined_14_5.h5
--------------------------------------------------
Predicting 5 days ahead with 30 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 1s 45ms/step
Evaluation Metrics:
Accuracy: 0.5344827586206896
Precision: 0.6415094339622641
Recall: 0.4927536231884058
F1 Score: 0.5573770491803278
Hit Rate: 0.5344827586206896
Average RoR: 0.3136145691623562
Model saved as combined_30_5.h5
--------------------------------------------------
Predicting 5 days ahead with 90 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9/9 [==============================] - 1s 71ms/step
Evaluation Metrics:
Accuracy: 0.5910780669144982
Precision: 0.6218905472636815
Recall: 0.7861635220125787
F1 Score: 0.6944444444444444
Hit Rate: 0.5910780669144982
Average RoR: 0.24657745286935298
Model saved as combined_90_5.h5
--------------------------------------------------
Predicting 30 days ahead with 90 days timeframe.


...

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 2s 226ms/step
Evaluation Metrics:
Accuracy: 0.7835497835497836
Precision: 0.7865168539325843
Recall: 0.9210526315789473
F1 Score: 0.8484848484848485
Hit Rate: 0.7835497835497836
Average RoR: 1.3833107141987515
Model saved as combined_365_30.h5
--------------------------------------------------
Predicting 90 days ahead with 365 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 2s 180ms/step
Evaluation Metrics:
Accuracy: 0.7748917748917749
Precision: 0.8198757763975155
Recall: 0.8516129032258064
F1 Score: 0.8354430379746834
Hit Rate: 0.7748917748917749
Average RoR: 2.9080678625233385
Model saved as combined_365_90.h5
--------------------------------------------------
Evaluation scores saved to 'combined_model_evaluation_scores.csv'.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F2_Score'])

# Take only the part of the data that was not used for training
split_index = int(len(X) * 0.7)
data = data[split_index:]

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order

            # Load model for timeframe and prediction
            model = load_model(f"models/combined_{timeframe}_{prediction}.h5")

            # Create a new array to store the combined predictions
            y_pred_combined = []
            y_val_combined = []

            # Iterate over all dates in the data (part of image filename)
            dates = data['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
            print(dates)
            for date in dates:
                # Filter the data based on the current date
                data = data[data['Image'].str.contains(str(date))]
                data = data.reset_index(drop=True)
            
                X_val = np.array(data['Image_Array'].tolist())
                y_val = data['Label']
                y_val_combined.append(y_val[0])
                
                # Evaluate the model
                y_pred = model.predict(X_val)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                # If y_pred_binary contains more 1s than 0s, then predict 1, else predict 0
                if np.sum(y_pred_binary) > len(y_pred_binary) / 2:
                    y_pred_combined.append(1)
                else:
                    y_pred_combined.append(0)
                
            accuracy = accuracy_score(y_val_combined, y_pred_combined)
            precision = precision_score(y_val_combined, y_pred_combined)
            recall = recall_score(y_val_combined, y_pred_combined)
            f2_score = fbeta_score(y_val_combined, y_pred_combined, beta=2)
        
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F2 Score: {f2_score}")
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F2_Score': f2_score
            }, ignore_index=True)
        
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_prediction_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_prediction_evaluation_scores.csv'.")